In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

%matplotlib inline

df =  pd.read_csv('data/articles_community.csv')
del df['Unnamed: 0']

# Show df to get an idea of the data
df.head()

,doc_body,doc_description,doc_full_name,doc_status,article_id
0,Skip navigation Sign in SearchLoading...\r\n\r...,Detect bad readings in real time using Python ...,Detect Malfunctioning IoT Sensors with Streami...,Live,0
1,No Free Hunch Navigation * kaggle.com\r\n\r\n ...,"See the forest, see the trees. Here lies the c...",Communicating data science: A guide to present...,Live,1
2,☰ * Login\r\n * Sign Up\r\n\r\n * Learning Pat...,Here’s this week’s news in Data Science and Bi...,"This Week in Data Science (April 18, 2017)",Live,2
3,"DATALAYER: HIGH THROUGHPUT, LOW LATENCY AT SCA...",Learn how distributed DBs solve the problem of...,DataLayer Conference: Boost the performance of...,Live,3
4,Skip navigation Sign in SearchLoading...\r\n\r...,This video demonstrates the power of IBM DataS...,Analyze NY Restaurant data using Spark in DSX,Live,4


In [2]:
# doc2vec, find topics of doc, keywords using name entity recognition, sentiment of the doc

In [3]:
import re

In [4]:
url_regex = re.compile('((http[s]?|www)://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)|(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+.com')

In [5]:
from emoji import UNICODE_EMOJI

def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            break
    return bool(count)

In [25]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [14]:
import string
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag

nltk.download(['stopwords', 'punkt', 'averaged_perceptron_tagger', 'wordnet'])

[nltk_data] Downloading package stopwords to /Users/Qian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Qian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Qian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/Qian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
def get_wordnet_tag(postag): 
    '''
    convert pos_tag to wordnet tag
    '''
    if postag.startswith('N'):
        return wordnet.NOUN
    if postag.startswith('J'):
        return wordnet.ADJ
    if postag.startswith('V'):
        return wordnet.VERB
    if postag.startswith('R'):
        return wordnet.ADV

In [28]:
def clean_doc(document, remove_stopwords=True):
    '''
    Clean document by
    1. set to lowercase
    2. replace url with placeholder
    3. replace emoji with placeholder
    4. remove punctuation and number
    5. tokenize document
    6. remove stop words
    7. pos-tag word
    8. word lemmatization
    Return the cleaned list of words
    
    INPUT:
    document(str): a string of article content
    
    OUTPUT:
    tokens(list): a list of words 
    '''
    url_regex = re.compile('((http[s]?|www)://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)|(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+.com')
    emoji_regex = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    # set to lowercase
    document = document.lower()
    # replace url with placeholder
    document = url_regex.sub('urlplaceholder', document) 
    # replace emoji with placeholder
    document = emoji_pattern.sub('emojiplaceholder', document)
    # remove punctuation and number
    document = re.sub(r"[^a-zA-Z]", " ", document)
    
#     # tokenize doc
#     tokens = word_tokenize(document)
#     # remove stopwords
#     if remove_stopwords:
#         tokens = [t for t in tokens if t not in stopwords.words('english')]
#     # word lemmatization
#     clean_tokens = []
#     lemm = WordNetLemmatizer()
#     for t, postag in pos_tag(tokens):
#         wordnet_tag = get_wordnet_tag(postag)
#         if wordnet_tag is not None:
#             clean_t = lemm.lemmatize(t, pos=wordnet_tag)
#         else:
#             clean_t = lemm.lemmatize(t)
        
#         clean_tokens.append(clean_t)
#     return clean_tokens

In [29]:
corpus = df.loc[0:10, 'doc_body'].values.tolist()

In [30]:
%timeit clean_corpus = [clean_doc(document) for document in corpus]

118 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
corpus= df.loc[0:100, 'doc_body']

In [20]:
%timeit clean_corpus = corpus.apply(clean_doc)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/Qian/anaconda/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-967725b8d929>", line 1, in <module>
    get_ipython().run_line_magic('timeit', 'clean_corpus = corpus.apply(clean_doc)')
  File "/Users/Qian/anaconda/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2287, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-61>", line 2, in timeit
  File "/Users/Qian/anaconda/envs/nlp/lib/python3.6/site-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/Users/Qian/anaconda/envs/nlp/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1131, in timeit
    time_number = timer.timeit(number)
  File "/Users/Qian/anaconda/envs/nlp/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 160, in tim

KeyboardInterrupt: 

In [185]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(clean_corpus)]

In [ ]:
documents[0]

In [188]:
model = Doc2Vec(vector_size=50, min_count=2, epochs=10)

In [190]:
model.build_vocab(documents)

In [196]:
model.wv.vocab['ibm'].count, model.corpus_count

(13, 3)

In [197]:
%time model.train(documents=documents, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 47.5 ms, sys: 153 µs, total: 47.7 ms
Wall time: 54.3 ms


In [198]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])

array([ 0.00637083, -0.01862202,  0.05612812, -0.00390257,  0.02477497,
       -0.02068868, -0.01358751,  0.0252157 , -0.05875756, -0.04738544,
        0.01031176,  0.07945748, -0.02274838, -0.02227109, -0.09419893,
        0.04544314,  0.05684213, -0.00726552, -0.01334044,  0.00240441,
        0.006685  , -0.02184754, -0.02439349, -0.05571279,  0.03009524,
       -0.06877074, -0.01022925,  0.06512682,  0.00485887,  0.02260521,
       -0.01859164, -0.05530703, -0.01326269, -0.00954779,  0.00321662,
        0.01423311, -0.06779686, -0.01815374, -0.0626078 , -0.01531944,
        0.03587846,  0.0785649 ,  0.03677639,  0.01420841,  0.04532422,
       -0.04057891,  0.04257825, -0.03085098,  0.00967675,  0.06713583], dtype=float32)

In [202]:
inferred_vec = model.infer_vector(documents[0].words)

In [203]:
model.docvecs.most_similar([inferred_vec], topn=3)

[(1, 0.9999544620513916), (2, 0.999953031539917), (0, 0.9999325275421143)]

In [ ]:
# use LDA to generate topics of each articles
# cluseter articles according to topics